In [1]:
#1: Gets all in-built function needed
import pandas as pd
import copy
import operator
import xlwt 
from xlwt import Workbook

In [2]:
#2: Get customer data in excel file into python dataframe
def getCustomerDataFilePath(in_customerDataFilePath):
    
    cusData = pd.read_excel(in_customerDataFilePath)
    return cusData

In [3]:
#3:Get path to file from user
#filePath = "C:\Users\User\Desktop\PejuWork\sample.xls"
customerDataFilePath=raw_input("Enter path to customer measurement data:")

Enter path to customer measurement data:C:\Users\User\Desktop\PejuWork\sample.xls


In [4]:
#4: extract customer data from dataframe, prepare a dictionary labelling each custormer from 0-99
def extractDataDictAndKeys(in_cusData):
    #Get number of rows in custormerData
    numberOfCustormer = len(in_cusData)
    
    custDict = {}
    
    #Create dictionary with keys numbering each persons measurement
    for a in range (0, numberOfCustormer):
        custDict[a] = []
        
    custormerDataList = in_cusData.values.tolist()
    
    customerDataKey = custDict.keys() 
    
    for i in customerDataKey:
        custDict[i]=custormerDataList[i]
        
    return custDict, customerDataKey

In [5]:
def toleranceToFloat(toleranceDimList):
    toleranceDimList = toleranceDimList.split(",")
    
    tolFloat=[]
    for val in toleranceDimList:
        tVal = float(val)
        tolFloat.append(tVal)
    
    return tolFloat

In [6]:
#Enter tolerance for each dimension of measurement e.g.3,2,5.08,5.08,5.08 
tolerance=toleranceToFloat(raw_input("Enter tolerance value for each dimension"))

Enter tolerance value for each dimension3,2,5.08,5.08,5.08


In [7]:
def compare_Xij_Xkj(xi, xk, in_tolerance):
    outpList=[]
    for xij, xkj, t in zip(xi, xk, in_tolerance):
        temp = abs(xij - xkj)
        if(temp<=t):
            outpList.append(1)
        else:
            outpList.append(0)
    return outpList 

In [8]:
def compute_wikj(in_inputData_Dict, in_tolerance):
    wikj_Dict ={}
    for key1, value1 in in_inputData_Dict.iteritems():
        xij_xik={}
        for key2, value2 in in_inputData_Dict.iteritems():
            temp = compare_Xij_Xkj(value1, value2, tolerance)
            xij_xik[key2]=temp
    
        wikj_Dict[key1]= xij_xik
    
    return wikj_Dict
             

In [9]:
def compute_matrixTbymatrix(m1):
    sumN=0
    for e1 in m1:
        sumN+=e1*e1
    
    return sumN

In [10]:
def compute_ir_Dict(in_B_ik):
    rik_All={}
    for key1, value1 in in_B_ik.iteritems():
        rik={}
        for key2, value2 in value1.iteritems():
            num = compute_matrixTbymatrix(value2)
            deNum=compute_matrixTbymatrix(value1[key1])
            rik_temp = float(float(num)/float(deNum))
            rik[key2]=rik_temp
        rik_All[key1]=rik
    return rik_All

In [11]:
def computer_Ri_ForAll_i(in_r_ik_ForAll_i):
    R_i_Dic={}
    for key1, value1 in in_r_ik_ForAll_i.iteritems():
    #for i in range(len(in_r_ik_ForAll_i)):
        R_i_sum=0
        for key2, value2 in value1.iteritems():
        #for k in range(len(in_r_ik_ForAll_i[i])):
            if(key1!=key2):
                R_i_sum+=value1[key2]
        #R_i_Dic[key1]=round(R_i_sum,2)
        R_i_Dic[key1]=round(R_i_sum,3)
    
    return R_i_Dic  

In [12]:
def compute_Ri_ideal(Dict):
    List=[]
    maxList=[]
    List=Dict.values()
    
    getMax=max(List)
    for key, value in Dict.iteritems():
    #for j in range(len(Dict)):
        if(value==getMax):
            maxList.append(key) 
            break
    return maxList

In [13]:
def returnCluster(in_rik_dict, in_R_i_Ideal):
    k_Dict={ }

    clusterList = []
    clusterList.append(in_R_i_Ideal[0])
    for key, value in in_rik_dict[in_R_i_Ideal[0]].iteritems():
        if(key!=in_R_i_Ideal[0]):
            k_Dict[key]=value

    k_val_List=k_Dict.values()        

    maxRel = max(k_val_List)

    for key, value in k_Dict.iteritems():
        if(value==maxRel):
            clusterList.append(key)
        
    return clusterList

In [14]:
def cluster_ri_rel(in_rik_dict, in_cluster):
    cluster_Rel_Dict={}
    for key, value in in_rik_dict.iteritems():
        r_i_rel = {}
        if(key in in_cluster):
            for key2, value2 in value.iteritems():
                if(key2 in in_cluster):
                    r_i_rel[key2]=value2
            cluster_Rel_Dict[key]=r_i_rel       

    return cluster_Rel_Dict

In [15]:
def getPercentageFit(in_ri_rel_cluster):
    sumNList=[]
    for key, value in in_ri_rel_cluster.iteritems():
        temp = value.values()
        tempSum= round(sum(temp),2)
        sumNList.append(tempSum)
    netSum = sum(sumNList)

    l=len(sumNList)
    netSum = netSum-l
    perFit = 100*((netSum)/(l*(l-1)))

    return round(perFit,2)

In [16]:
def computerAggLossIndex(in_clusterList,in_custDict):
    sumOut =[]
    result=-1
    i= in_clusterList[0]
    #i_index=in_clusterList[0]
    t_sum=0
    sumL=[]
    for k in in_clusterList:
        sum2=0
        if (k!=i):
            #k_index=in_clusterList[i]
            
            for v1, v2 in zip (in_custDict[k], in_custDict[i]):
                sum2+=(v1- v2)**2
            
        sumL.append(sum2)
    t_sum=(sum(sumL))**0.5
    if (len(in_clusterList)!=0):
        result = (t_sum/len(in_clusterList))

    return round(result,2)

In [17]:
def getAggLoss(in_cluster, in_custDict, in_R_i_Ideal):
    netSumN = []
    for value in in_cluster:
        sumN = 0
        for v1, v2 in zip (in_custDict[in_R_i_Ideal[0]], in_custDict[value]):
            sumN+= (v1-v2)**2
    
        sumN =sumN**0.5
        netSumN.append(sumN)

    
    result =  (sum(netSumN))/len(netSumN)
    
    return round(result,3)

In [18]:
def getLastCluster (in_R_i_ForAll_i):
    listLastCluster=[]
    for key, value in in_R_i_ForAll_i.iteritems():
        listLastCluster.append(key)

    return listLastCluster

In [19]:
def resetData(in_cluster, in_custDict):
    newDict = {}
    for key, value in in_custDict.iteritems():
        if (key not in in_cluster):
            
            newDict[key]=value
            
            
    return newDict    

In [20]:
def recordData(in_cluster,in_percentageFit,in_aggLoss):
    records=["ClusterKey", "DegreeFit", "AggLoss"]
    clusterDict = {}
    for a in records:
        clusterDict[a] = []

    clusterDict["ClusterKey"]=in_cluster
    clusterDict["DegreeFit"]=in_percentageFit
    clusterDict["AggLoss"]= in_aggLoss
    
    return clusterDict

In [21]:
def tryRun(in_custDict, in_tolerance, in_start):
    result = {}
    Bik_dict = compute_wikj(in_custDict, in_tolerance)
    rik_dict = compute_ir_Dict(Bik_dict)
    R_i_ForAll_i = computer_Ri_ForAll_i(rik_dict)
    R_i_Ideal = compute_Ri_ideal(R_i_ForAll_i)
    cluster = returnCluster(rik_dict, R_i_Ideal)
    ri_rel_cluster = cluster_ri_rel(rik_dict, cluster)
    percentageFit = getPercentageFit(ri_rel_cluster)   
    aggLoss = getAggLoss(cluster, in_custDict, R_i_Ideal)
    
    result = recordData(cluster,percentageFit,aggLoss)
    
    data = resetData(cluster, in_custDict)
   
    return result, data

In [22]:
def LastRun(in_custDict, in_tolerance, in_start):
    result = {}
    Bik_dict = compute_wikj(in_custDict, in_tolerance)
    rik_dict = compute_ir_Dict(Bik_dict)
    R_i_ForAll_i = computer_Ri_ForAll_i(rik_dict)
    R_i_Ideal = compute_Ri_ideal(R_i_ForAll_i)
    cluster = getLastCluster (R_i_ForAll_i)
    ri_rel_cluster = cluster_ri_rel(rik_dict, cluster)
    percentageFit = getPercentageFit(ri_rel_cluster)   
    aggLoss = getAggLoss(cluster, in_custDict, R_i_Ideal)
    
    result = recordData(cluster,percentageFit,aggLoss)
    
    data = resetData(cluster, in_custDict)
   
    return result, data

In [23]:
def runRecursive(in_custDict, in_tolerance, in_start, cluster_num, in_result):
    
    getTest, dataTest = tryRun(in_custDict, in_tolerance, in_start)
    if (in_start==cluster_num):
        getTest, dataTest=LastRun(in_custDict, in_tolerance, in_start)
        in_result[in_start]=getTest
    else:
        in_result[in_start]=getTest
        in_start+=1
        runRecursive(dataTest, in_tolerance, in_start, cluster_num, in_result)
        
    return in_result

In [24]:
def LabelToList(labelRaw):
    labels = labelRaw.split(",")
    
    return labels

In [25]:
def getOutputToExcel(in_clusters, in_OutputLabel, in_outFilePath):

    wb = Workbook() 
    for key, value in clusters.iteritems():
        outputSheet = wb.add_sheet("sheet"+str(key))
        setSheet(outputSheet, OutputLabel)
        for j in range(len(value)):
            outputSheet.write(j+1, 0, j+1)
            for i in range(len(value[j])):
            #print j, "sheet"+str(key), value[j][i]
                outputSheet.write(j+1, i+1, value[j][i])
        outputSheet.write(1, len(OutputLabel)+2, degFit[key])
        outputSheet.write(1, len(OutputLabel)+3, aggLoss[key])
                

    #wb.save('C:/Users/User/Desktop/PejuWork/test2.xls')
    wb.save(in_outFilePath)

In [26]:
def setResult(in_result, in_inputData_Dict):
    ClusterDict={}
    AggLossDict={}
    DegreeFitDict={}
    for key, value in in_result.iteritems():
        clusterList=[]
        for key2, value2 in value.iteritems():
            if (key2=="ClusterKey"):
                for i in value2:
                    clusterList.append(in_inputData_Dict[i])
    
            if (key2=="DegreeFit"):
                DegreeFitDict[key] = value2
            if (key2=="AggLoss"):
                AggLossDict[key]=value2
        ClusterDict[key]=clusterList

    return ClusterDict,DegreeFitDict,AggLossDict

In [27]:
def setSheet(sheet, in_dimLabel):
    for i in range(len(in_dimLabel)):
        sheet.write(0, i, in_dimLabel[i]) 
     
    sheet.write(0, len(in_dimLabel)+2, 'DegreeFit')
    sheet.write(0, len(in_dimLabel)+3, 'AggLoss')

In [28]:
cusData=getCustomerDataFilePath(customerDataFilePath)

In [29]:
custDict,customerDataKey = extractDataDictAndKeys(cusData)
start = 1
result={}

In [30]:
#result, data = tryRun(custDict, tolerance, start)

In [31]:
result = runRecursive(custDict, tolerance, 1, 5, result)

In [32]:
#print result

In [33]:
#Enter the lables for each dimension of measurement, separated by (,):
#e.g. 'Length(cm)','Waist(cm)','Thigh(cm)','Hip(cm)','Bottom girth(cm)' 
OutputLabel=LabelToList(raw_input("Enter labels for each dimension"))

Enter labels for each dimension'Length(cm)','Waist(cm)','Thigh(cm)','Hip(cm)','Bottom girth(cm)'


In [34]:
clusters, degFit,aggLoss = setResult(result, custDict) 

In [35]:
#Enter path for your output file: where your output will be put in excel
#e.g. C:\Users\User\Desktop\PejuWork\test2.xls
outFilePath=raw_input("Enter path to customer measurement data:")

Enter path to customer measurement data:C:\Users\User\Desktop\PejuWork\test5.xls


In [36]:
getOutputToExcel(clusters, OutputLabel, outFilePath)